# 02_06_legacywrap.ipynb - Wrapping legacy code with Cython, CFFI, and F2PY

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as pp

#### Code from 02_03_numpycompute.ipynb

In [ ]:
def initphi(n=64):
    dx = 1/n
    xs = np.linspace(0.5*dx, 1-0.5*dx, n)
    
    phi = np.zeros((n,n), 'd')
    
    phi[:,-1] = np.sin(2 * math.pi * xs)
    phi[-1,:] = -np.sin(2 * math.pi * xs)
    
    return phi

In [ ]:
def showphi(array, colorbar=True):
    pp.imshow(array.T, origin='lower', extent=(0,1,0,1),
              vmin=-1, vmax=1, cmap='coolwarm')
    
    if colorbar:
        pp.colorbar()

#### Wrapping C with Cython

In [ ]:
%%file gauss_iterate.c

#define A(i,j) (array[(i)*nx + (j)])

void gauss_iterate(int nx, int ny, double *array, int iterations) {
    for(int k=0; k<iterations; k++) {
        for(int i=1; i<nx-1; i++) {
            for(int j=1; j<ny-1; j++) {
                A(i,j) = (A(i-1,j) + A(i+1,j) + A(i,j-1) + A(i,j+1)) / 4;
            }
        }
    }
}

In [ ]:
%%file gauss_iterate.h

void gauss_iterate(int nx, int ny, double *array, int iterations);

In [ ]:
%%file cgauss.pyx

# cython: language_level=3

cdef extern from "gauss_iterate.h":
    cdef void gauss_iterate(int nx, int ny, double array[], int iterations)

# it would be even safer to declare array with "double [:,::1]",
# which would accept only contiguous arrays

def gauss(double [:,:] array, int iterations):
    # array is a Cython "memoryview", but it supports numpy
    # operations such as obtaining its shape
    cdef int nx = array.shape[0], ny = array.shape[1]
    
    # call the C function, obtaining the address of the first array item
    gauss_iterate(nx, ny, &array[0,0], iterations)

In [ ]:
%%file setup.py

from setuptools import setup, Extension
from Cython.Build import cythonize

ext_modules = [
    Extension("cgauss",
              sources=["cgauss.pyx", "gauss_iterate.c"],
              includes=["gauss_iterate.h"]
              )
]

setup(name="gauss",
      ext_modules=cythonize(ext_modules))

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import cgauss

In [ ]:
cgauss.gauss

In [ ]:
phi = initphi(128)

In [ ]:
cgauss.gauss(phi, 2000)

In [ ]:
showphi(phi)

#### Wrapping C with CFFI

In [ ]:
# on OS X or Linux (including WSL)
!gcc -I. gauss_iterate.c -shared -o gauss_iterate.so

In [ ]:
# on Windows with Visual Studio 2019
# open the x64 Native Tools Command Prompt
# navigate to Ch02 folder

# cl /LD gauss_iterate.c

In [ ]:
from cffi import FFI

In [ ]:
ffi = FFI()

In [ ]:
ffi.cdef("void gauss_iterate(int nx, int ny, double *array, int iterations);")

In [ ]:
cfgauss = ffi.dlopen('./gauss_iterate.so')

# on Windows with Visual Studio 2019
# cfgauss = ffi.dlopen('gauss_iterate.dll')

In [ ]:
cfgauss

In [ ]:
phi = initphi(128)

In [ ]:
cfgauss.gauss_iterate(128, 128, ffi.cast("double *", phi.ctypes.data), 2000)

In [ ]:
showphi(phi)

#### Wrapping Fortran with f2py

In [ ]:
%%file gauss_iterate.f90

subroutine gauss(array,nx,ny,iterations)
    implicit none

    real*8, dimension(0:nx-1,0:ny-1), intent(inout) :: array
    integer, intent(in)                             :: nx,ny,iterations

    integer k,i,j
            
    do k=1,iterations
        do i=1,nx-2
            do j=1,ny-2
                array(i,j) = (array(i-1,j) + array(i+1,j) + array(i,j-1) + array(i,j+1)) / 4
            end do
        end do
    end do

    return
end subroutine gauss

In [ ]:
!f2py3 -m fgauss -c gauss_iterate.f90

In [ ]:
import fgauss

In [ ]:
?fgauss.gauss

In [ ]:
fphi = np.asfortranarray(initphi(128))

In [ ]:
fgauss.gauss(fphi, 2000)

In [ ]:
showphi(fphi)

#### Wrapping Fortran with fortran-magic

In [ ]:
# !pip install fortran-magic
%load_ext fortranmagic

In [ ]:
%%fortran

subroutine fgauss(array,nx,ny,iterations)
    implicit none

    real*8, dimension(0:nx-1,0:ny-1), intent(inout) :: array
    integer, intent(in)                             :: nx,ny,iterations

    integer k,i,j
            
    do k=1,iterations
        do i=1,nx-2
            do j=1,ny-2
                array(i,j) = (array(i-1,j) + array(i+1,j) + array(i,j-1) + array(i,j+1)) / 4
            end do
        end do
    end do

    return
end subroutine fgauss

In [ ]:
fgauss

In [ ]:
?fgauss